In [1]:
import pandas as pd
from tqdm.auto import tqdm, trange

In [53]:
# read file C:/Users/samue/Downloads/input.bin
data = b''
with open('C:/Users/samue/Downloads/input.bin', 'rb') as f:
    data = f.read()
len(data)

166404

In [49]:
dists = {}
ordered = []
current = 0
for b in tqdm(data):
    if b:
        if current in dists:
            dists[current] += 1
        else:
            dists[current] = 1
        ordered.append(current)
        current = 0
    else:
        current += 1
dists

  0%|          | 0/166404 [00:00<?, ?it/s]

{0: 277,
 19: 1,
 7: 5283,
 15: 2565,
 23: 1287,
 31: 683,
 39: 306,
 47: 151,
 55: 77,
 71: 15,
 63: 42,
 16: 4,
 79: 13,
 8: 7,
 22: 1,
 87: 4,
 14: 5,
 119: 1,
 95: 2,
 24: 5,
 103: 3,
 6: 6,
 40: 1,
 30: 1}

In [4]:
df = pd.DataFrame.from_dict(dists, orient='index', columns=['count'])
ordered = pd.Series(ordered)
data = pd.Series(data)
ordered.head()

0     0
1    19
2     7
3     7
4    15
dtype: int64

In [5]:
ordered.describe()

count    10740.000000
mean        14.493203
std         11.318566
min          0.000000
25%          7.000000
50%          7.000000
75%         15.000000
max        119.000000
dtype: float64

probabilmente se guardo solo gruppi da 8 byte posso vedere qualcosa


In [11]:
s = pd.Series([int(x) for x in data[4:]])
s

0         0
1         0
2         0
3         0
4         0
         ..
166395    0
166396    0
166397    0
166398    0
166399    0
Length: 166400, dtype: int64

In [34]:
# s is a bit serie
# I want to have bytes
byte_l = []
for i in trange(0, len(s), 8):
    byte = 0
    for j in range(8):
        byte += s[i+j] * 2 ** j
    byte_l.append(byte)
byte = pd.Series(byte_l)
byte

  0%|          | 0/20800 [00:00<?, ?it/s]

0          0
1          0
2          1
3          1
4          1
        ... 
20795      0
20796      1
20797      1
20798      1
20799    170
Length: 20800, dtype: int64

In [35]:
byte = byte[byte <= 255]
byte

0          0
1          0
2          1
3          1
4          1
        ... 
20795      0
20796      1
20797      1
20798      1
20799    170
Length: 20765, dtype: int64

In [40]:
#samue think again
wrap = []
for i in trange(0, int(len(byte)/8)*8, 8):
    tmp = 0
    for j in range(8):
        tmp+= byte.iloc[i+j] * 2 ** j
    wrap.append(tmp)
wrap = pd.Series(wrap)
wrap

  0%|          | 0/2595 [00:00<?, ?it/s]

0       220
1       215
2       204
3        69
4        78
       ... 
2590    202
2591     21
2592    107
2593    192
2594     41
Length: 2595, dtype: int64

In [45]:
wrap = wrap[wrap <= 255]

In [46]:
# write to file C:/Users/samue/Downloads/output.bin
b = bytearray([x for x in wrap.to_numpy()])
b = bytes(b)
with open('C:/Users/samue/Downloads/output.bin', 'wb') as f:
    f.write(b)

In [54]:
data = pd.Series([int(x) for x in data])

In [57]:
cos = data[data != 0][data != 255][data != 1]

In [58]:
cos

0          18
2596      205
5196       64
7796        5
10396     245
         ... 
158597      3
161196    242
163796     84
163797      3
166396    170
Length: 95, dtype: int64